In [2]:
from pyopenms import *
import pandas as pd
import numpy as np
import os
import glob

input_mzml_files = glob.glob('results/interim/*.mzML')

raw_data=[]
for filename in input_mzml_files:
    exp= MSExperiment()
    MzMLFile().load(filename, exp)
    exp.sortSpectra(True)
    raw_data.append(exp)

ref_index = [i[0] for i in sorted(enumerate([exp.size() for exp in raw_data]), key=lambda x:x[1])][-1]
aligner = MapAlignmentAlgorithmPoseClustering()
aligner_par= aligner.getDefaults()
aligner_par.setValue("max_num_peaks_considered", 3000)
aligner.setParameters(aligner_par)
aligner.setReference(raw_data[ref_index])

for exp in raw_data[:ref_index] + raw_data[ref_index+1:]:
    trafo = TransformationDescription()
    aligner.align(exp, trafo)
    transformer = MapAlignmentTransformer()
    transformer.transformRetentionTimes(exp, trafo, True)

for exp in raw_data:    
    mzml_file = os.path.join("results", "", "FFMI", "", "interim", "", 'MapAligned_' + os.path.basename(exp.getLoadedFilePath())[19:-5] +".mzML")
    MzMLFile().store(mzml_file, exp)

Determination of memory status is not supported on this 
 platform, measuring for memoryleaks will never fail


In [6]:
from pyopenms import *
import pandas as pd
import numpy as np
import os
import glob
import csv
# read tsv file and create list of FeatureFinderMetaboIdentCompound
def metaboTableFromFile(path_to_library_file):
    metaboTable = []
    with open(path_to_library_file, 'r') as tsv_file:
        tsv_reader = csv.reader(tsv_file, delimiter="\t")
        next(tsv_reader) # skip header
        for row in tsv_reader:
            metaboTable.append(FeatureFinderMetaboIdentCompound(
                row[0], # name
                row[1], # sum formula
                float(row[2]), # mass
                [int(charge) for charge in row[3].split(',')], # charges
                [float(rt) for rt in row[4].split(',')], # RTs
                [float(rt_range) for rt_range in row[5].split(',')], # RT ranges
                [float(iso_distrib) for iso_distrib in row[6].split(',')] # isotope distributions
            ))
    return metaboTable

In [14]:
input_mzml_files=glob.glob("results/FFMI/interim/*.mzML")
# load ms data from mzML file into MSExperiment
for mzml_file in input_mzml_files:
    spectra = MSExperiment()
    MzMLFile().load(mzml_file, spectra)

    # create FeatureFinderAlgorithmMetaboIdent and assign ms data
    ff = FeatureFinderAlgorithmMetaboIdent()
    ff.setMSData(spectra)

    # read library generate a metabo table with compounds
    metabo_table = metaboTableFromFile('epemicins.tsv')

    # FeatureMap to store results
    fm = FeatureMap()
    params = ff.getParameters()
    params[b'extract:mz_window'] = 5.0 
    params[b'detect:peak_width'] = 20.0  #adjust for wide peaks
    ff.setParameters(params)

    # run the FeatureFinderMetaboIdent with the metabo_table and store results in fm
    ff.run(metabo_table, fm)

    # save FeatureMap to file
    ff_file = os.path.join("results", "", "FFMI", "", "interim", "", 'FFMI_' + os.path.basename(mzml_file)[11:-5] +".featureXML")
    FeatureXMLFile().store(ff_file, fm)

RT window size calculated as 240 seconds.
RT window size calculated as 80 seconds.
Extracting chromatograms...
Detecting chromatographic peaks...
Found 16 feature candidates in total.
7 features left after selection of best candidates.
4 features left after resolving overlaps (involving 5 features in 1 groups).
Model fitting: 4 successes, 0 failures
Imputing model failures with a linear model based on log(rawIntensities). Slope: 0.956071, Intercept: 0.733679
RT window size calculated as 240 seconds.
RT window size calculated as 80 seconds.


Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 1423.830750294896006.
Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 712.418518849583506.
Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 712.418579884739756.


Extracting chromatograms...
Detecting chromatographic peaks...
Found 4 feature candidates in total.
4 features left after selection of best candidates.
1 features left after resolving overlaps (involving 4 features in 1 groups).
Model fitting: 1 successes, 0 failures
Imputing model failures with a linear model based on log(rawIntensities). Slope: 1, Intercept: -0.0315086
RT window size calculated as 240 seconds.
RT window size calculated as 80 seconds.
Extracting chromatograms...
Detecting chromatographic peaks...
Found 11 feature candidates in total.
7 features left after selection of best candidates.
3 features left after resolving overlaps (involving 6 features in 1 groups).
Model fitting: 3 successes, 0 failures
Imputing model failures with a linear model based on log(rawIntensities). Slope: 0.957582, Intercept: 0.657689
RT window size calculated as 240 seconds.
RT window size calculated as 80 seconds.
Extracting chromatograms...
Detecting chromatographic peaks...
Found 11 feature 

Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 712.418762990208506.


Extracting chromatograms...
Detecting chromatographic peaks...
Found 0 feature candidates in total.
0 features left after selection of best candidates.
No overlaps between features found.
RT window size calculated as 240 seconds.
RT window size calculated as 80 seconds.


Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 712.418824025364756.
Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 1423.830750294896006.
Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 712.418457814427256.
<Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 712.418824025364756.> occurred 2 times
Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z t

Extracting chromatograms...
Detecting chromatographic peaks...
Found 11 feature candidates in total.
7 features left after selection of best candidates.
3 features left after resolving overlaps (involving 6 features in 1 groups).
Model fitting: 3 successes, 0 failures
Imputing model failures with a linear model based on log(rawIntensities). Slope: 0.966574, Intercept: 0.586134
RT window size calculated as 240 seconds.
RT window size calculated as 80 seconds.
Extracting chromatograms...
Detecting chromatographic peaks...
Found 16 feature candidates in total.
7 features left after selection of best candidates.
3 features left after resolving overlaps (involving 5 features in 1 groups).
Model fitting: 3 successes, 0 failures
Imputing model failures with a linear model based on log(rawIntensities). Slope: 0.818958, Intercept: 3.05602
RT window size calculated as 240 seconds.
RT window size calculated as 80 seconds.
Extracting chromatograms...
Detecting chromatographic peaks...
Found 16 fea

Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 712.418762990208506.
Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 712.418824025364756.
Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 1423.830750294896006.
Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 712.418457814427256.
<Found no signal. The Gaussian width is probably smaller than the spacing in your chromatogram data. Try to use a bigger width. The error occurred in the chromatogram with m/z time 712.4188240253

RT window size calculated as 80 seconds.
Extracting chromatograms...
Detecting chromatographic peaks...
Found 0 feature candidates in total.
0 features left after selection of best candidates.
No overlaps between features found.


In [15]:
from collections import defaultdict
from functools import reduce
from pathlib import Path
from time import perf_counter
import sys

from IPython.core.display import display
from pandas import CategoricalDtype
import numpy as np
from pyopenms import *
import pandas as pd
import os

common_meta_value_types = {
    b'label': 'U30',
    b'spectrum_index': 'i',
    b'score_fit': 'f',
    b'score_correlation': 'f',
    b'FWHM': 'f',
    b'spectrum_native_id': 'U30',
    b"num_of_masstraces" : "f"
}

class FeatureMapDF(FeatureMap):
    def __init__(self):
        super().__init__()
    
    # meta_values = None (default), 'all' or list of meta value names
    def get_df(self, meta_values = None):
        # get all possible meta value keys in a set
        if meta_values == 'all':
            meta_values = set()
            for f in self:
                mvs = []
                f.getKeys(mvs)
                for m in mvs:
                    meta_values.add(m)
        elif not meta_values: # if None, set to empty list
            meta_values = []
        
        def gen(fmap: FeatureMap, fun):
            for f in fmap:
                yield from fun(f, meta_values)

        def extract_meta_data(f: Feature, meta_values):
            bb = f.getConvexHull().getBoundingBox2D()
                
            vals = [f.getMetaValue(m) if f.metaValueExists(m) else np.NA for m in meta_values]   # find some NA or None value for numpy

            yield f.getUniqueId(), f.getCharge(), f.getRT(), f.getMZ(), bb[0][0], bb[1][0], f.getOverallQuality(), f.getIntensity()

        cnt = self.size()

        mddtypes = [('id', np.dtype('uint64')), ('charge', 'i4'), ('RT', 'f'), ('mz', 'f'),
                    ('RTstart', 'f'), ('RTend', 'f'),
                    ('quality', 'f'), ('intensity', 'f')]
        
        for meta_value in meta_values:
            if meta_value in common_meta_value_types:
                mddtypes.append((meta_value.decode(), common_meta_value_types[meta_value]))
            else:
                mddtypes.append((meta_value.decode(), 'U50'))
        mdarr = np.fromiter(iter=gen(self, extract_meta_data), dtype=mddtypes, count=cnt)
        df= pd.DataFrame(mdarr).set_index('id').sort_values("mz")
        #df= df[df["num_of_masstraces"]>=2]
        return df

In [16]:
input_feature_files = glob.glob('results/FFMI/interim/*.featureXML')

for filename in input_feature_files:
    fmap = FeatureMapDF()
    FeatureXMLFile().load(filename, fmap)
    DF= fmap.get_df()
    feature_csv= os.path.join("results", "", "FFMI", "", 'features_' + os.path.basename(filename)[:-10] +"csv")
    DF.to_csv(feature_csv)
    print(os.path.basename(filename))
    display(DF)

FFMI_20210827_UMETAB219_POS_ISP2_Plate-2_MDNAWGS14_rep3.featureXML


,charge,RT,mz,RTstart,RTend,quality,intensity
id,,,,,,,
898219834071531883,2,415.619019,712.418457,408.922302,422.787354,3.666471,1.072226e+07
3460016777711225559,2,390.762238,712.418762,379.194702,399.325195,4.049423,4.156025e+07
3477270708278713665,1,406.521027,1423.830811,400.505493,408.922302,3.391791,3.418768e+05


FFMI_20210827_UMETAB219_POS_DNPM_Plate-2_MDNAWGS14_rep3.featureXML


,charge,RT,mz,RTstart,RTend,quality,intensity
id,,,,,,,
10181117891572555396,2,406.219849,712.418457,393.602020,416.997620,4.537503,2.410169e+07
9484657296525451682,2,392.352997,712.418518,385.452759,393.602020,1.127428,3.371586e+05
18000097019771845006,1,406.080200,1423.830811,397.157074,415.829895,4.534957,1.636310e+06


FFMI_20210827_UMETAB219_POS_ISP2_Plate-2_MDNAWGS14_blank.featureXML


...

,charge,RT,mz,RTstart,RTend,quality,intensity
id,,,,,,,
18366424970370854982,2,406.608856,712.418457,396.079559,421.341766,4.670281,32199670.0
15314620205705370511,2,392.294403,712.418762,386.787445,396.079559,1.774502,1123933.0
12244745190026934049,1,406.573364,1423.830811,397.248871,420.891846,4.803393,2324051.0


FFMI_20210827_UMETAB219_POS_FPY12_Plate-2_MDNAWGS14_rep3.featureXML


,charge,RT,mz,RTstart,RTend,quality,intensity
id,,,,,,,
12760000848910850311,2,391.063904,712.418762,380.524353,396.193268,2.610967,6680591.0
10108628872995354404,2,405.428864,712.418823,396.193268,418.096100,4.376209,45885900.0
13089269739129057234,1,405.287506,1423.830811,395.008575,416.872986,4.406720,4211494.0


FFMI_20210827_UMETAB219_POS_FPY12_Plate-2_MDNAWGS14_blank.featureXML


,charge,RT,mz,RTstart,RTend,quality,intensity
id,,,,,,,


In [21]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d

path= 'results/FFMI/interim/*.featureXML'

def plotDetectedFeatures3D(filename):
    fm = FeatureMap()
    fh = FeatureXMLFile()
    fh.load(filename, fm)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    for feature in fm:
        color = next(ax._get_lines.prop_cycler)['color']
        # chromatogram data is stored in the subordinates of the feature
        for i, sub in enumerate(feature.getSubordinates()):
            retention_times = [x[0] for x in sub.getConvexHulls()[0].getHullPoints()]
            intensities = [int(y[1]) for y in sub.getConvexHulls()[0].getHullPoints()]
            mz = sub.getMetaValue('MZ')
            ax.plot(retention_times, intensities, zs = mz, zdir = 'x', color = color)
            if i == 0:
                ax.text(mz,retention_times[0], max(intensities)*1.02, feature.getMetaValue('label'), color = color)

    ax.set_ylabel('time (s)')
    ax.set_xlabel('m/z')
    ax.set_zlabel('intensity (cps)')
    plt.show()


In [22]:
for filename in path:
    plotDetectedFeatures3D(filename)

NameError: name 'path_to_featureXML' is not defined